In [59]:
# Import Packages
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from datetime import date, timedelta
from pycaret import time_series as ts
from pycaret.time_series import TSForecastingExperiment
from pycaret.datasets import get_data
#import refinitiv.data as rd
from os.path import exists

# Only enable critical logging (Optional)
import os
os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"

# Global Figure Settings for notebook ----
global_fig_settings = {"renderer": "notebook", "width": 1000, "height": 600}

In [60]:
dx = dx = pd.read_csv(r'/home/nkem/Documents/PhD_Research/allN11Oct2022.csv')
dx['incidentdate'] = pd.to_datetime(dx['incidentdate'])
td = dx.copy()
dk = td.groupby([pd.Grouper(key='incidentdate', freq='M')])['estimatedqty'].agg(['sum','size'])
dk = dk.reset_index()
dk.rename(columns={"sum":"estimatedqty", "size":"spillno"}, inplace=True)

df = dk[["incidentdate","spillno"]]
df= df.reset_index()

df["month"] = df["incidentdate"].dt.month
df["year"] = df["incidentdate"].dt.year
dt = df[["incidentdate","spillno","month", "year"]]
dt = dt.set_index("incidentdate")
#target = df[["incidentdate","spillno"]]

#dt = dt.set_index("incidentdate")
#target = target.set_index("incidentdate")

data = dt.loc[: "2022-09-30"]
#y_train = target.loc[: "2021-12-31"]
#X_valid = dt.loc["2022-01-31" : "2022-06-30"]
#y_valid = target.loc["2022-01-31" : "2022-06-30"]

#X_train = X_train.values
#y_train = y_train.values

In [61]:
target = "spillno"
exog_vars = ['year', 'month']
include = [target] + exog_vars
data = data[include]
data.tail()

,spillno,year,month
incidentdate,,,
2022-05-31,22,2022,5
2022-06-30,48,2022,6
2022-07-31,20,2022,7
2022-08-31,60,2022,8
2022-09-30,38,2022,9


In [62]:
exp_auto = TSForecastingExperiment()

# enforce_exogenous=False --> Use multivariate forecasting when model supports it, else use univariate forecasting
exp_auto.setup(
    data=data, target=target, fh=3, enforce_exogenous=False,
    numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
    fig_kwargs=global_fig_settings, session_id=42,use_gpu=False
)

,Description,Value
0,session_id,42
1,Target,spillno
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(213, 3)"
5,Transformed data shape,"(213, 3)"
6,Transformed train set shape,"(210, 3)"
7,Transformed test set shape,"(3, 3)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [63]:
metric = "mae"

In [64]:
# Include slower models like Prophet (turbo=False), but exclude some specific models ----
best = exp_auto.compare_models(sort=metric, turbo=False)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.2797,0.2752,5.9010,8.0316,0.1961,0.2188,-0.0144,0.1600
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.3093,0.2641,6.5344,7.7174,0.2588,0.2484,0.0476,1.5767
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.3124,0.2426,6.6142,7.0994,0.2888,0.2540,-0.1071,0.1333
snaive,Seasonal Naive Forecaster,0.3149,0.2555,6.6667,7.4769,0.2699,0.2467,-0.1710,0.7200
tbats,TBATS,0.3255,0.2654,6.8733,7.7553,0.2576,0.2602,0.0525,7.5267
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.3317,0.2891,7.0038,8.4437,0.2695,0.2611,-0.0477,0.3200
auto_arima,Auto ARIMA,0.3319,0.3226,7.0066,9.4195,0.2490,0.2575,-0.2258,17.4633
bats,BATS,0.3371,0.2960,7.1295,8.6558,0.2607,0.2702,-0.3275,6.0033
theta,Theta Forecaster,0.3396,0.2904,7.1748,8.4828,0.2648,0.2673,-0.0645,0.0633
exp_smooth,Exponential Smoothing,0.3622,0.3096,7.6492,9.0420,0.2821,0.2854,-0.1631,0.2200


Processing:   0%|          | 0/137 [00:00<?, ?it/s]

In [65]:
from pycaret.datasets import get_data
from pycaret.classification import *

In [66]:
data

,spillno,year,month
incidentdate,,,
2005-01-31,3,2005,1
2005-02-28,1,2005,2
2005-03-31,2,2005,3
2005-04-30,32,2005,4
2005-05-31,2,2005,5
...,...,...,...
2022-05-31,22,2022,5
2022-06-30,48,2022,6
2022-07-31,20,2022,7


In [67]:
from pycaret.time_series import *

In [68]:
s = setup(data, target = 'spillno')

,Description,Value
0,session_id,3833
1,Target,spillno
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(213, 3)"
5,Transformed data shape,"(213, 3)"
6,Transformed train set shape,"(212, 3)"
7,Transformed test set shape,"(1, 3)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [69]:
best = compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
grand_means,Grand Means Forecaster,0.7395,0.5311,15.3213,15.3213,0.6387,0.3807,0.0267
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.8873,0.6372,18.3734,18.3734,0.6709,0.4509,0.0667
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,0.9116,0.6546,18.8763,18.8763,0.4001,0.4898,0.1167
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.9940,0.7137,20.5653,20.5653,0.4224,0.5620,0.0733
par_cds_dt,Passive Aggressive w/ Cond. Deseasonalize & Detrending,1.0242,0.7354,21.1850,21.1850,0.5450,0.5982,0.0867
croston,Croston,1.0301,0.7397,21.3315,21.3315,0.4661,0.5733,0.1933
snaive,Seasonal Naive Forecaster,1.0311,0.7403,21.3333,21.3333,0.4722,0.5900,0.0367
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.0776,0.7738,22.3104,22.3104,0.5717,0.5972,0.2033
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.1760,0.8445,24.3468,24.3468,0.6956,0.6371,0.1100
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.1837,0.8500,24.5092,24.5092,0.6774,0.6490,0.2533


Processing:   0%|          | 0/109 [00:00<?, ?it/s]

In [70]:
plot_model(best)

In [71]:
dt = create_model(best)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2022-05,0.2795,0.2007,5.8038,5.8038,0.1209,0.1140
1,2022-06,1.6300,1.1707,33.7762,33.7762,1.6888,0.9156
2,2022-07,0.3089,0.2218,6.3839,6.3839,0.1064,0.1124
Mean,NaT,0.7395,0.5311,15.3213,15.3213,0.6387,0.3807
SD,NaT,0.6298,0.4524,13.0517,13.0517,0.7426,0.3783


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [72]:
tuned_dt = tune_model(best)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2022-05,0.2795,0.2007,5.8038,5.8038,0.1209,0.1140
1,2022-06,1.6300,1.1707,33.7762,33.7762,1.6888,0.9156
2,2022-07,0.3089,0.2218,6.3839,6.3839,0.1064,0.1124
Mean,NaT,0.7395,0.5311,15.3213,15.3213,0.6387,0.3807
SD,NaT,0.6298,0.4524,13.0517,13.0517,0.7426,0.3783


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished


In [73]:
top3 = compare_models(n_select = 3)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
grand_means,Grand Means Forecaster,0.7395,0.5311,15.3213,15.3213,0.6387,0.3807,0.0333
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.8873,0.6372,18.3734,18.3734,0.6709,0.4509,0.0667
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,0.9116,0.6546,18.8763,18.8763,0.4001,0.4898,0.1100
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.9940,0.7137,20.5653,20.5653,0.4224,0.5620,0.0700
par_cds_dt,Passive Aggressive w/ Cond. Deseasonalize & Detrending,1.0242,0.7354,21.1850,21.1850,0.5450,0.5982,0.0700
croston,Croston,1.0301,0.7397,21.3315,21.3315,0.4661,0.5733,0.1700
snaive,Seasonal Naive Forecaster,1.0311,0.7403,21.3333,21.3333,0.4722,0.5900,0.0267
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.0776,0.7738,22.3104,22.3104,0.5717,0.5972,0.2133
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.1760,0.8445,24.3468,24.3468,0.6956,0.6371,0.1167
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.1837,0.8500,24.5092,24.5092,0.6774,0.6490,0.2400


Processing:   0%|          | 0/111 [00:00<?, ?it/s]

In [74]:
best_model = compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
grand_means,Grand Means Forecaster,0.7395,0.5311,15.3213,15.3213,0.6387,0.3807,0.0267
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.8873,0.6372,18.3734,18.3734,0.6709,0.4509,0.0633
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,0.9116,0.6546,18.8763,18.8763,0.4001,0.4898,0.1200
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.9940,0.7137,20.5653,20.5653,0.4224,0.5620,0.0700
par_cds_dt,Passive Aggressive w/ Cond. Deseasonalize & Detrending,1.0242,0.7354,21.1850,21.1850,0.5450,0.5982,0.0633
croston,Croston,1.0301,0.7397,21.3315,21.3315,0.4661,0.5733,0.1867
snaive,Seasonal Naive Forecaster,1.0311,0.7403,21.3333,21.3333,0.4722,0.5900,0.0267
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.0776,0.7738,22.3104,22.3104,0.5717,0.5972,0.2367
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.1760,0.8445,24.3468,24.3468,0.6956,0.6371,0.1000
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.1837,0.8500,24.5092,24.5092,0.6774,0.6490,0.2500


Processing:   0%|          | 0/109 [00:00<?, ?it/s]

In [75]:
from pycaret.datasets import get_data
from pycaret.regression import *

In [76]:
best_model

NaiveForecaster(sp=1, strategy='mean', window_length=None)

In [79]:
# import classification module
from pycaret.regression import *

# init setup
clf1 = setup(data, target = target)

# train a decision tree model
dt = create_model('dt')

# tune hyperparameters of decision tree
tuned_dt = tune_model(dt)

# tune hyperparameters with increased n_iter
tuned_dt = tune_model(dt, n_iter = 50)

# tune hyperparameters to optimize AUC
tuned_dt = tune_model(dt) #default is 'Accuracy'

# tune hyperparameters with custom_grid
params = {"max_depth": np.random.randint(1, (len(data.columns)*.85),20),
          "max_features": np.random.randint(1, len(data.columns),20),
          "min_samples_leaf": [2,3,4,5,6],
          "criterion": ["gini", "entropy"]
          }

tuned_dt_custom = tune_model(dt, custom_grid = params)

# tune multiple models dynamically
top3 = compare_models(n_select = 3)
tuned_top3 = [tune_model(i) for i in top3]

,Description,Value
0,Session id,4818
1,Target,spillno
2,Target type,Regression
3,Data shape,"(213, 3)"
4,Train data shape,"(149, 3)"
5,Test data shape,"(64, 3)"
6,Numeric features,2
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,11.0667,198.9333,14.1044,0.3152,0.3031,0.2669
1,16.2667,759.0667,27.5512,-0.6956,0.3331,0.2948
2,15.1333,356.0667,18.8697,0.7502,0.3269,0.2681
3,15.8667,825.0667,28.7240,0.2320,0.3403,0.3204
4,10.8667,175.2667,13.2388,0.8565,0.2369,0.1819
5,12.8000,202.2667,14.2220,0.7243,0.2964,0.3137
6,9.2000,126.4000,11.2428,0.8754,0.2040,0.1809
7,20.7333,1049.1333,32.3903,0.2462,0.3847,0.3202
8,12.0000,235.0667,15.3319,0.7365,0.3838,0.2403


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.8875,163.2622,12.7774,0.4380,0.2737,0.2423
1,17.0441,535.3962,23.1386,-0.1960,0.5201,0.5066
2,23.8119,778.2910,27.8979,0.4540,0.4550,0.4327
3,18.7061,593.0776,24.3532,0.4479,0.4317,0.3984
4,18.9408,660.4565,25.6993,0.4593,0.3479,0.2855
5,12.3842,275.9757,16.6125,0.6238,0.5240,0.6103
6,18.7268,632.6274,25.1521,0.3762,0.3316,0.3157
7,23.3847,1050.3139,32.4085,0.2454,0.4714,0.5115
8,16.6455,435.6553,20.8724,0.5116,0.6596,0.8434


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:19:52
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Decision Tree Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


KeyError: 'entropy'

In [ ]:
# tune hyperparameters with custom_grid
params = {"max_depth": np.random.randint(1, (len(data.columns)*.85),20),
          "max_features": np.random.randint(1, len(data.columns),20),
          "min_samples_leaf": [2,3,4,5,6],
          "criterion": ["gini", "entropy"]
          }


In [77]:
tuned_best_model_hyperopt = tune_model(
    "best_model",
    search_library="tune-sklearn",
    search_algorithm="grid",
    n_iter=20
)


TypeError: The behavior of tune_model in version 1.0.1 is changed. Please pass trained model object.

In [82]:
from pycaret.datasets import get_data
from pycaret.regression import *

#data = get_data("house")
exp = setup(
    data,
    target = target,
    #test_data=test, # so that the entire dataset is used for cross validation - do not normally do this!
    session_id=42,
    fold=5
)
rf = create_model("rf")
en = create_model("en")

tune_model(rf, search_library = "tune-sklearn", search_algorithm="hyperopt", n_iter=20)
tune_model(rf, search_library = "tune-sklearn", search_algorithm="optuna", optimize="RMSLE", n_iter=20)

tune_model(en, search_library = "tune-sklearn", search_algorithm="hyperopt", n_iter=20)
tune_model(en, search_library = "tune-sklearn", search_algorithm="optuna", optimize="RMSLE", n_iter=20)


(<ray.tune.trainable.util._Trainable object at 0x7f0736687d90> pid=1130079) /home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/resource_tracker.py:311: UserWarning: resource_tracker: There appear to be 7 leaked semlock objects to clean up at shutdown
(<ray.tune.trainable.util._Trainable object at 0x7f0736687d90> pid=1130079)   'resource_tracker: There appear to be '
(<ray.tune.trainable.util._Trainable object at 0x7feac6bddf10> pid=1129910) /home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/resource_tracker.py:311: UserWarning: resource_tracker: There appear to be 7 leaked semlock objects to clean up at shutdown
(<ray.tune.trainable.util._Trainable object at 0x7feac6bddf10> pid=1129910)   'resource_tracker: There appear to be '
(<ray.tune.trainable.util._Trainable object at 0x7f03befddd10> pid=1129963) /home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib

(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) 
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) 
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) LokyProcess-5 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)

(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) exception calling callback for <Future at 0x7f8f5c913410 state=finished raised TerminatedWorkerError>
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 26, in _invoke_callbacks
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)     callback(self)
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/parallel.py", line 385, in __call__
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)     self.parallel.dispatch_next()
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)   File "

(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) 
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) 
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) LokyProcess-1 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0> pid=1132234)

RayTaskError(TerminatedWorkerError): [36mray::_Inner.train()[39m (pid=1132234, ip=192.168.1.106, repr=<ray.tune.trainable.util._Trainable object at 0x7f8f62e47cd0>)
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/ray/tune/trainable/trainable.py", line 355, in train
    raise skipped from exception_cause(skipped)
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/ray/tune/trainable/trainable.py", line 352, in train
    result = self.step()
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/tune_sklearn/_trainable.py", line 111, in step
    return self._train()
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/tune_sklearn/_trainable.py", line 252, in _train
    error_score="raise")
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 283, in cross_validate
    for train, test in cv.spl

In [85]:
exp = setup(
    data,
    target = "spillno",
    session_id=42,
    fold=5
)

rf = create_model("rf")

tune_model(rf, search_library = "tune-sklearn", search_algorithm="optuna", optimize="RMSLE", n_iter=20) # Will run on Ray cluster!


(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) 
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) 
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) LokyProcess-4 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)

(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) exception calling callback for <Future at 0x7fb11951d850 state=finished raised TerminatedWorkerError>
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 26, in _invoke_callbacks
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)     callback(self)
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/parallel.py", line 385, in __call__
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)     self.parallel.dispatch_next()
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)   File "

(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) 
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) 
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) LokyProcess-6 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0> pid=1134934)

RayTaskError(TerminatedWorkerError): [36mray::_Inner.train()[39m (pid=1134934, ip=192.168.1.106, repr=<ray.tune.trainable.util._Trainable object at 0x7fb11fa87dd0>)
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/ray/tune/trainable/trainable.py", line 355, in train
    raise skipped from exception_cause(skipped)
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/ray/tune/trainable/trainable.py", line 352, in train
    result = self.step()
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/tune_sklearn/_trainable.py", line 111, in step
    return self._train()
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/tune_sklearn/_trainable.py", line 252, in _train
    error_score="raise")
  File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 283, in cross_validate
    for train, test in cv.spl

(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071) 
(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071) 
(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071) LokyProcess-3 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7fb91ba8be10> pid=1135071)

...

(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) 
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) 
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) LokyProcess-1 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)

(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) exception calling callback for <Future at 0x7f4e940e8990 state=finished raised TerminatedWorkerError>
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 26, in _invoke_callbacks
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)     callback(self)
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/parallel.py", line 385, in __call__
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)     self.parallel.dispatch_next()
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)   File "

(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) 
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) 
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) LokyProcess-3 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7f4e9a669090> pid=1135186)

(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) exception calling callback for <Future at 0x7f363c9a3d50 state=finished raised TerminatedWorkerError>
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 26, in _invoke_callbacks
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104)     callback(self)
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/parallel.py", line 385, in __call__
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104)     self.parallel.dispatch_next()
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104)   File "

(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) 
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) 
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) LokyProcess-7 failed with traceback: 
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) --------------------------------------------------------------------------------
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104) Traceback (most recent call last):
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104)   File "/home/nkem/miniconda3/envs/pycaret_timeseries/lib/python3.7/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 164, in <module>
(<ray.tune.trainable.util._Trainable object at 0x7f3642e87d90> pid=1135104)